In [3]:
import torch

def native_softmax(x):
    x_max = x.max(dim=1)[0]

    z = x - x_max[:,None]

    e = torch.exp(z)
    sum = e.sum(dim=1)

    return e/sum[:,None]


x = torch.Tensor([[2,9],[54,1]])
print(native_softmax(x))

tensor([[9.1105e-04, 9.9909e-01],
        [1.0000e+00, 9.6027e-24]])


In [ ]:
import torch

import triton
import triton.language as tl

# 这里的row_idx = tl.program_id(axis=0)表示当前程序的ID，而row_step = tl.num_programs(0)则表示总的程序数量。
# 比如说，如果你有一个1000行的矩阵，而GPU上启动了100个并行程序，那么：
# tl.num_programs(0) 返回100
# 每个程序需要处理10行数据(1000/100)
# 每个程序通过其program_id来确定它应该处理哪些行
# 这种设计的优点是：
# 实现了数据的并行处理
# 确保了工作负载的均匀分配
# 避免了线程之间的数据竞争
# 在实际应用中，这个值通常用于：
# 计算每个线程需要处理的数据量
# 确定数据的步进(stride)大小
# 实现负载均衡
